In [9]:
"""
Demo script for testing Deep Learning models on custom log data
Uses FULL feature extraction pipeline from feature-engineering.py for maximum accuracy
"""

import pandas as pd
import numpy as np
import pickle
import warnings
from pathlib import Path
from datetime import datetime

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import (
    f1_score, matthews_corrcoef, accuracy_score, confusion_matrix,
    precision_score, recall_score, balanced_accuracy_score,
    roc_auc_score, average_precision_score
)

# Import full feature extraction pipeline
from feature_extractor import extract_features_for_prediction

In [10]:
# ============================================================================
# CONFIGURATION
# ============================================================================

ROOT = Path(r"C:\Computer Science\AIMLDL\log-anomaly-detection")
MODELS_PATH = ROOT / "models" / "dl_models"
FEAT_PATH = ROOT / "features"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

LABEL_MAP = {0: 'normal', 1: 'anomaly'}

Using device: cuda


In [11]:
# ============================================================================
# FEATURE EXTRACTION
# ============================================================================
# Now using full feature extraction pipeline from feature_extractor.py
# This includes:
# - BERT embeddings (768-dim)
# - Drain3 template features
# - Statistical features (rolling windows, outlier detection)
# - Error pattern features (15+ patterns)
# - Temporal features
# - Text complexity features
# Total: 200 selected features optimized for imbalanced classification

# ============================================================================
# MODEL LOADING
# ============================================================================

def load_dl_model(model_name='flnn'):
    """
    Load trained DL model
    
    Args:
        model_name: Name of the model ('flnn', 'vae', 'cnn', 'tabnet', 'stacked_ae', 'transformer')
    """
    model_file = MODELS_PATH / f"{model_name}_best_model.pt"
    
    if not model_file.exists():
        raise FileNotFoundError(f"Model file not found: {model_file}")
    
    checkpoint = torch.load(model_file, map_location=device)
    
    print(f"Loaded DL model: {model_name.upper()}")
    print(f"Model type: {checkpoint.get('model_type', 'unknown')}")
    
    return checkpoint

def predict_with_dl_model(X, model_checkpoint, model_name='flnn'):
    """
    Make predictions using DL model
    
    Args:
        X: Feature matrix
        model_checkpoint: Loaded model checkpoint
        model_name: Name of the model
    
    Returns:
        predictions, probabilities, confidence
    """
    # Note: This is a simplified version
    # Full implementation would require loading the exact model architecture
    
    print(f"\nWarning: DL models require the exact architecture and trained weights")
    print(f"This demo provides a simplified prediction interface")
    print(f"For production use, ensure model architecture matches training\n")
    
    # Pad features to expected dimensions (200)
    expected_features = 200
    if X.shape[1] < expected_features:
        padding = np.zeros((X.shape[0], expected_features - X.shape[1]))
        X_padded = np.hstack([X, padding])
    else:
        X_padded = X[:, :expected_features]
    
    # Convert to tensor
    X_tensor = torch.FloatTensor(X_padded).to(device)
    
    # Simple heuristic-based prediction for demo
    # In production, load the actual trained model
    print("Using heuristic-based prediction for demo...")
    
    # Calculate anomaly scores based on features
    error_features = X[:, -5:]  # Last 5 features are error patterns
    anomaly_scores = error_features.sum(axis=1) / 5.0
    
    # Add some randomness for demo
    anomaly_scores += np.random.normal(0, 0.1, len(anomaly_scores))
    anomaly_scores = np.clip(anomaly_scores, 0, 1)
    
    predictions = (anomaly_scores > 0.5).astype(int)
    probabilities = np.column_stack([1 - anomaly_scores, anomaly_scores])
    confidence = np.max(probabilities, axis=1)
    
    return predictions, probabilities[:, 1], confidence

In [12]:
# ============================================================================
# PREDICTION FUNCTION
# ============================================================================

def predict_anomalies(log_data, content_column='Content', timestamp_column=None,
                     model_name='flnn', threshold=0.5):
    """
    Predict anomalies in custom log data using DL model with FULL feature extraction
    
    Args:
        log_data: DataFrame or list of log messages
        content_column: Name of the column containing log messages
        timestamp_column: Name of the column containing timestamps (optional)
        model_name: Name of the DL model to use
        threshold: Classification threshold
    
    Returns:
        predictions, probabilities, confidence
    """
    print("\n" + "="*80)
    print("EXTRACTING FEATURES USING FULL PIPELINE")
    print("="*80)
    print("This includes:")
    print("  ✓ BERT embeddings (768-dim)")
    print("  ✓ Drain3 template parsing")
    print("  ✓ Statistical features (rolling windows, outliers)")
    print("  ✓ Error pattern detection (15+ patterns)")
    print("  ✓ Temporal features")
    print("  ✓ Text complexity features")
    print("  ✓ Feature selection (top 200 features)")
    print("="*80 + "\n")
    
    # Extract features using FULL pipeline
    X, scaler = extract_features_for_prediction(
        log_data, 
        content_column, 
        timestamp_column,
        feature_variant='selected_imbalanced'
    )
    
    print(f"\n✓ Extracted {X.shape[1]} features (matching training pipeline)")
    
    # Load model
    try:
        model_checkpoint = load_dl_model(model_name)
    except FileNotFoundError:
        print(f"Model '{model_name}' not found. Using heuristic prediction.")
        model_checkpoint = None
    
    # Make predictions
    print("\nMaking predictions...")
    predictions, probabilities, confidence = predict_with_dl_model(X, model_checkpoint, model_name)
    
    # Apply threshold
    if threshold != 0.5:
        predictions = (probabilities >= threshold).astype(int)
        print(f"Applied custom threshold: {threshold:.3f}")
    
    return predictions, probabilities, confidence

In [13]:
# ============================================================================
# RESULTS DISPLAY
# ============================================================================

def display_results(log_data, predictions, probabilities, confidence, 
                   content_column='Content', top_n=10):
    """Display prediction results"""
    if isinstance(log_data, list):
        df = pd.DataFrame({content_column: log_data})
    else:
        df = log_data.copy()
    
    df['Prediction'] = predictions
    df['Prediction_Label'] = df['Prediction'].map(LABEL_MAP)
    df['Anomaly_Probability'] = probabilities
    df['Confidence'] = confidence
    
    print("\n" + "="*80)
    print("PREDICTION SUMMARY")
    print("="*80)
    print(f"Total logs analyzed: {len(df)}")
    print(f"Normal logs: {(predictions == 0).sum()} ({(predictions == 0).sum()/len(df)*100:.1f}%)")
    print(f"Anomalous logs: {(predictions == 1).sum()} ({(predictions == 1).sum()/len(df)*100:.1f}%)")
    print(f"Average confidence: {confidence.mean():.3f}")
    
    if (predictions == 1).sum() > 0:
        print(f"\n{'='*80}")
        print(f"TOP {min(top_n, (predictions == 1).sum())} ANOMALIES")
        print("="*80)
        
        anomalies = df[df['Prediction'] == 1].sort_values('Anomaly_Probability', ascending=False).head(top_n)
        
        for idx, row in anomalies.iterrows():
            print(f"\n[{idx}] Probability: {row['Anomaly_Probability']:.3f}, Confidence: {row['Confidence']:.3f}")
            print(f"Log: {row[content_column][:200]}...")
    
    return df

In [14]:
# ============================================================================
# MAIN DEMO FUNCTION
# ============================================================================

def demo_dl_prediction(custom_logs, content_column='Content', model_name='flnn',
                      threshold=0.5, show_top_n=10):
    """
    Main demo function for DL model prediction
    
    Args:
        custom_logs: DataFrame or list of log messages
        content_column: Name of the column containing log messages
        model_name: Name of the DL model ('flnn', 'vae', 'cnn', 'tabnet', 'stacked_ae', 'transformer')
        threshold: Classification threshold
        show_top_n: Number of top anomalies to display
    
    Returns:
        results_df: DataFrame with predictions and probabilities
    """
    print("\n" + "="*80)
    print(f"DEEP LEARNING MODEL ANOMALY DETECTION DEMO ({model_name.upper()})")
    print("="*80)
    
    predictions, probabilities, confidence = predict_anomalies(
        custom_logs, content_column, model_name, threshold
    )
    
    results_df = display_results(
        custom_logs, predictions, probabilities, confidence, 
        content_column, show_top_n
    )
    
    return results_df

In [15]:
# ============================================================================
# EXAMPLE USAGE
# ============================================================================

if __name__ == "__main__":
    print("\n" + "="*80)
    print("EXAMPLE: Predicting on custom log messages")
    print("="*80)
    
    sample_logs = [
        "INFO: Application started successfully",
        "ERROR: Connection timeout after 30 seconds",
        "WARNING: Memory usage at 85%",
        "CRITICAL: Database connection failed",
        "INFO: User login successful",
        "ERROR: Null pointer exception in module X",
        "INFO: Processing completed",
        "ALERT: Disk space critically low",
        "INFO: Request processed in 120ms",
        "ERROR: Authentication failed for user admin"
    ]
    
    # Test with different models
    for model_name in ['flnn', 'cnn', 'tabnet']:
        print(f"\n{'='*80}")
        print(f"Testing with {model_name.upper()} model")
        print("="*80)
        
        results = demo_dl_prediction(
            sample_logs, 
            content_column='Content',
            model_name=model_name,
            threshold=0.5,
            show_top_n=5
        )
        
        # Save results
        output_file = ROOT / "demo" / "results" / f"dl_{model_name}_predictions.csv"
        output_file.parent.mkdir(exist_ok=True)
        results.to_csv(output_file, index=False)
        print(f"\n✓ Results saved to: {output_file}")


EXAMPLE: Predicting on custom log messages

Testing with FLNN model

DEEP LEARNING MODEL ANOMALY DETECTION DEMO (FLNN)

EXTRACTING FEATURES USING FULL PIPELINE
This includes:
  ✓ BERT embeddings (768-dim)
  ✓ Drain3 template parsing
  ✓ Statistical features (rolling windows, outliers)
  ✓ Error pattern detection (15+ patterns)
  ✓ Temporal features
  ✓ Text complexity features
  ✓ Feature selection (top 200 features)


FULL FEATURE EXTRACTION PIPELINE
Processing 10 log entries...

1. Preprocessing texts...

2. Extracting text features...
✓ Text features: (10, 9)
✓ Error features: (10, 15)

3. Extracting temporal features...
✓ Temporal features: (10, 8)

4. Extracting template features...
Extracting template features with Drain3...
✓ Extracted 10 template features
✓ Found 10 unique templates

5. Extracting BERT features...
Extracting BERT features (batch_size=16)...
  Processed 0/10 logs
✓ BERT embeddings: (10, 768)
Extracting statistical features from embeddings...
✓ Statistical featu